In [32]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [33]:
np.set_printoptions(precision=2)

In [34]:
ratings = pd.read_csv(r'Dataset/ratings.csv', sep=",")
movies = pd.read_csv(r'Dataset/movies_metadata.csv', sep=",", low_memory=False)[['id','title']]


In [35]:
print(movies[-10:])

           id                      title
45456   84419           House of Horrors
45457  390959  Shadow of the Blair Witch
45458  289923        The Burkittsville 7
45459  222848            Caged Heat 3000
45460   30840                 Robin Hood
45461  439050                     Subdue
45462  111109        Century of Birthing
45463   67758                   Betrayal
45464  227506           Satan Triumphant
45465  461257                   Queerama


In [36]:
ratings.drop(['timestamp'],axis=1, inplace=True)


In [40]:
len(movies['id'])
len(ratings)

26024289

In [38]:
movies['id'] = movies['id'].apply(lambda x: int(x) if x.isnumeric() else -1)


In [30]:
# movies['id'].drop(['1997-08-20'], axis=0)
# movies = movies[movies['id'] != '1997-08-20']

movies = movies[movies['id'] != -1]


AttributeError: 'int' object has no attribute 'isnumeric'

In [16]:
# user_item_matrix = ratings.pivot(index='movieId', columns='userId', values='rating')
data1 = ratings.iloc[0:1000000]
user_item_matrix = data1.pivot(index='movieId', columns='userId', values='rating',)

# data2 = ratings[100001:260000]
# user_item_matrix2 = data2.pivot(index='movieId', columns='userId', values='rating')
# user_item_matrix2
# user_item_matrix.shape

# user_item_matrix.head()
# array = np.array(ratings['userId'] < 0)
# if True in array :
#     print('yes')

In [17]:
user_item_matrix.fillna(0, inplace=True)


In [18]:
users_votes = data1.groupby('userId')['rating'].agg('count')
users_votes

userId
1        27
2        22
3        10
4        62
5        26
         ..
10179     5
10180    35
10181    18
10182    24
10183    62
Name: rating, Length: 10183, dtype: int64

In [19]:
movies_votes = data1.groupby('movieId')['rating'].agg('count')
movies_votes

movieId
1         2573
2         1004
3          577
4          101
5          598
          ... 
175975       1
176165       1
176211       2
176219       1
176271       1
Name: rating, Length: 20291, dtype: int64

In [24]:
# Фильтр
user_mask = users_votes[users_votes > 50].index
movie_mask = movies_votes[movies_votes > 10].index
# user_mask
movie_mask

Index([     1,      2,      3,      4,      5,      6,      7,      8,      9,
           10,
       ...
       168252, 168254, 168366, 168612, 169864, 169984, 170705, 170875, 171763,
       174055],
      dtype='int64', name='movieId', length=7029)

In [25]:
# применим фильтры и отберем фильмы с достаточным количеством оценок
user_item_matrix = user_item_matrix.loc[movie_mask, :]

# а также активных пользователей
user_item_matrix = user_item_matrix.loc[:,user_mask]

In [26]:
user_item_matrix.shape

(7029, 3953)

In [42]:
csr_data = csr_matrix(user_item_matrix.values)
print(csr_data[:2,:5])

  (0, 0)	1.0
  (0, 3)	4.0
  (0, 4)	4.5
  (1, 0)	2.0


In [43]:
user_item_matrix = user_item_matrix.rename_axis(None, axis = 1).reset_index()
zW

,index,movieId,4,7,8,9,11,12,15,16,...,10159,10162,10163,10165,10168,10169,10170,10175,10176,10183
0,0,1,0.0,0.0,4.0,4.5,0.0,4.0,0.0,0.0,...,2.0,3.0,4.5,2.0,3.0,4.0,4.5,4.5,0.0,3.0
1,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.5,0.0,0.0,2.0,2.5,4.0,0.0,0.0
2,2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7024,7024,169984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7025,7025,170705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7026,7026,170875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7027,7027,171763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
Knn = NearestNeighbors(metric= 'cosine', algorithm='brute', n_neighbors=20, n_jobs= -1)


NameError: name 'csr_data' is not defined

In [ ]:
Knn.fit(csr_data)

In [16]:
recommendation = 10
# search_word = 'Matrix'
search_word = input('Введите фильм : ')

In [17]:
movie_search = movies[movies['title'].str.contains(search_word, na=False)]
movie_search



,id,title
2458,603,The Matrix
6221,604,The Matrix Reloaded
6783,605,The Matrix Revolutions
29468,174615,Return to Source: The Philosophy of The Matrix
38306,21769,Armitage: Dual Matrix
43870,14543,The Matrix Revisited


In [18]:
movie_id = movie_search.iloc[0]['id']
movie_id

603

In [19]:
distances, indices = Knn.kneighbors(csr_data[int(movie_id)], n_neighbors = recommendation + 1)

In [20]:
indices

array([[ 603,  624,   68,  610,  656,  644, 1197, 1037,  669,   83,  426]],
      dtype=int64)

In [21]:
distances

array([[0.  , 0.6 , 0.63, 0.65, 0.65, 0.65, 0.66, 0.66, 0.66, 0.66, 0.67]])

In [22]:
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()

indices_distances = list(zip(indices_list, distances_list))

print(type(indices_distances[0]))


print(indices_distances[:3])

<class 'tuple'>
[(603, 0.0), (624, 0.5983569194199887), (68, 0.6266809855435538)]


In [23]:
indices_distances_sorted = sorted(indices_distances, key = lambda x: x[1], reverse = False)

indices_distances_sorted = indices_distances_sorted[1:]
indices_distances_sorted

[(624, 0.5983569194199887),
 (68, 0.6266809855435538),
 (610, 0.6455706131760419),
 (656, 0.6471664856054475),
 (644, 0.6541142882913827),
 (1197, 0.6554201796970729),
 (1037, 0.6624393635246741),
 (669, 0.6628986456267292),
 (83, 0.6646550816172356),
 (426, 0.6680124768731386)]

In [24]:
recom_list = []


for ind_dist in indices_distances_sorted:
    matrix_movie_id = user_item_matrix.iloc[ind_dist[0]]['movieId']
    id = movies[movies['id'] == matrix_movie_id].index
    if movies[movies['id'] == matrix_movie_id].empty:
        continue
    else:
        title = movies.iloc[id]['title'].values[0] 

    dist = ind_dist[1]

    recom_list.append({'Title' : title, 'Distance' : dist})
print(recom_list)
    
    

[{'Title': 'Almost Famous', 'Distance': 0.5983569194199887}, {'Title': 'Before Sunrise', 'Distance': 0.6266809855435538}, {'Title': 'The Wing or the Thigh', 'Distance': 0.6455706131760419}, {'Title': 'Krull', 'Distance': 0.6471664856054475}, {'Title': 'M', 'Distance': 0.6541142882913827}, {'Title': "Antonia's Line", 'Distance': 0.6628986456267292}, {'Title': 'Armageddon', 'Distance': 0.6646550816172356}, {'Title': 'Shaft in Africa', 'Distance': 0.6680124768731386}]
